In [ ]:
# read it in to inspect it
with open('tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [40]:
print("length of dataset in characters: ", len(text))
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)
print(chars[1])

length of dataset in characters:  1115394

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65
 


In [3]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [5]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

/home/medlar/SpikingReservoir/.venv/lib/python3.13/site-packages/torch/_subclasses/functional_tensor.py:295: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


torch.Size([1115394]) torch.int64


In [6]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

---
We now need to make characters into spike trains

---

In [ ]:
import numpy as np

def generate_spike_trains(input_data, U_max, U_min, N_sample):
    """
    Generate spike trains using Poisson encoding.
    
    Parameters:
        input_data (numpy.ndarray): Normalized input data (e.g., between U_min and U_max).
        U_max (float): Maximum value of the input range.
        U_min (float): Minimum value of the input range.
        N_sample (int): Number of spike samples.
    
    Returns:
        spike_trains (numpy.ndarray): Generated spike trains (0s and 1s).
    """
    # Calculate average spike interval
    h_k = N_sample * (U_max - input_data) / (U_max - U_min)
    
    # Generate intervals using Poisson distribution
    intervals = np.random.poisson(h_k)
    
    # Generate spike trains
    spike_trains = np.zeros((len(input_data), N_sample))
    for i, interval in enumerate(intervals):
        spike_indices = np.cumsum(np.random.choice(np.arange(1, N_sample + 1), interval, replace=False))
        spike_indices = spike_indices[spike_indices < N_sample]
        spike_trains[i, spike_indices] = 111
    
    return spike_trains


# Example input data
input_data = np.array([0.1, 0.4, 0.7, 0.9])  # Normalized input
U_max = 1.0
U_min = 0.0
N_sample = 100  # Spike sampling times

# Generate spike trains
spikes = generate_spike_trains(input_data, U_max, U_min, N_sample)
print("Generated Spike Trains:")
print(spikes)


In [ ]:
# SIMPLEST ENCODING
# each character activate 1 neuron, fixed

def char_to_spike_train(char):

    N_sam = 10
    spikes = np.zeros((N_sam, 65))
    for i in range(N_sam):
        spikes[i, encode(char)] = 1

    return spikes

print(char_to_spike_train("\n"))

---
Create the reservoire

N_res is the number of neurons in the reservoir

---

In [ ]:
N_res = 1000

